DATA CLEANING PIPELINE FOR SENTIMENTAL ANALYSIS

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import io
from google.colab import files
import torch

In [2]:
pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 12.6 MB/s eta 0:00:00


In [3]:
pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 10.6 MB/s eta 0:00:00


In [4]:
pip install demoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.8 MB/s eta 0:00:00


In [5]:
import emoji
import demoji

In [6]:
import contractions

In [7]:
uploaded = files.upload()                       #this function imports file from local machine
filename = list(uploaded.keys())[0]             #which element of the list is needed
data = pd.read_csv(io.BytesIO(uploaded[filename]))

Saving sentiment1000entrie.csv to sentiment1000entrie.csv


In [8]:
data.head()

,TARGET,ID,DATE,FLAG,USERNAME,TEXT
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [9]:
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   TARGET    1000 non-null   int64 
 1   ID        1000 non-null   int64 
 2   DATE      1000 non-null   object
 3   FLAG      1000 non-null   object
 4   USERNAME  1000 non-null   object
 5   TEXT      1000 non-null   object
dtypes: int64(2), object(4)
memory usage: 47.0+ KB


,TARGET,ID
count,1000.0,1.000000e+03
mean,0.0,1.467936e+09
std,0.0,7.154601e+04
min,0.0,1.467810e+09
25%,0.0,1.467875e+09
50%,0.0,1.467936e+09
75%,0.0,1.467998e+09
max,0.0,1.468056e+09


In [10]:
import spacy            #these both libraries are used for nlp and data mining or cleaning
import re
nlp = spacy.load('en_core_web_sm')   #loading the model
                                     #u can also add command that manages missing texts in the column

In [11]:
def urls(text):
  text = re.sub(r'http\S+|www\S+|https\S+|<.@#$%&*?>|[^a-zA-Z\s]', '', text)   #middle one is the replacement char which null here
  return text
data['clean_text'] = data['TEXT'].apply(lambda x: urls(x))

In [12]:
def emojis_to_text(text, delimiters=(" _"," _")):      #delimeters help preserve  emoji context
  return emoji.demojize(text, delimiters=delimiters)        #Also custom labelling of emojis can be done
data['clean_text'] = data['TEXT'].apply(lambda x: emojis_to_text(x))

In [13]:
def lemmetize(text):
  doc = nlp(text)                             #tokenization of text
  lemmas = [token.lemma_ for token in doc]    #tokenization is necessary here for proper extraction of text
  return ' '.join(lemmas)
data['clean_text'] = data['TEXT'].apply(lambda x: lemmetize(x))

In [14]:
SLANG_MAP = {
    "brb": "be right back",
    "ttyl": "talk to you later",
    "lol": "laugh out loud",
    "omg": "oh my god",
    "btw": "by the way",
    "idk": "i don't know",
    "smh": "shaking my head",
    "np": "no problem",
    "afaik": "as far as i know",
    "imo": "in my opinion",
    "dm": "direct message",
    "tbh": "to be honest",
    "irl": "in real life",
    "fyi": "for your information",
    "asap": "as soon as possible"
}

In [15]:
def contextual_slang_replacement(text):
  doc = nlp(text)
  replaced_tokens = []
  for token in doc:
    if token.text.lower() in SLANG_MAP:
      replaced_tokens.append(SLANG_MAP[token.text.lower()])
    else:
        replaced_tokens.append(token.text)
  return ' '.join(replaced_tokens)
data['clean_text'] = data['TEXT'].apply(lambda x: contextual_slang_replacement(x))

In [16]:
def handle_negations_spacy(text):
    doc = nlp(text)
    negation_tokens = [token for token in doc if token.dep_ == "neg"]

    # Create a list to store the modified tokens
    modified_tokens = []

    for token in doc:
        if token.dep_ in ("acomp", "advmod", "attr") and any(token.head == neg_token.head for neg_token in negation_tokens):
            modified_tokens.append(token.text + "_NOT")  # Append modified text
        else:
            modified_tokens.append(token.text)  # Append original text

    return " ".join(modified_tokens)  # Join modified tokens

data['clean_text'] = data['TEXT'].apply(lambda x: handle_negations_spacy(x))

In [17]:
def expand_contractions(text):
    return contractions.fix(text)
data['clean_text'] = data['TEXT'].apply(lambda x: expand_contractions(x))

In [18]:
def remove_stopwords_spacy(text, custom_stopwords=None):
  doc = nlp(text)
  stop_words = nlp.Defaults.stop_words
  filtered_tokens = [
      token.text for token in doc
      if not token.is_stop and token.is_alpha
  ]
  return ''.join(filtered_tokens)
data['clean_text'] = data['TEXT'].apply(lambda x: remove_stopwords_spacy(x))

In [19]:
def low_space(text):
  low = text.lower()
  space = low.replace(' ','')
  return space
data['clean_text'] = data['TEXT'].apply(lambda x: low_space(x))